In [40]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd

BASE_URL = "https://www.fantasypros.com/nfl/adp/half-point-ppr-overall.php"

def make_adp_df():
    res = requests.get(BASE_URL)
    if res.ok:
        soup = BS(res.content, 'html.parser')
        table = soup.find('table', {'id': 'data'})
        df = pd.read_html(str(table))[0]
        print('Output after reading the html:\n\n', df.head(), '\n') # so you can see the output at this point
        df = df[['Player Team (Bye)', 'POS', 'AVG']]
        print('Output after filtering:\n\n', df.head(), '\n')
        df['PLAYER'] = df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split()[:-2])) # removing the team and position
        df['POS'] = df['POS'].apply(lambda x: x[:2]) # removing the position rank
        
        df = df[['PLAYER', 'POS', 'AVG']].sort_values(by='AVG')
        
        print('Final output: \n\n', df.head())
        
        return df
        
    else:
        print('oops, something didn\'t work right', res.status_code)
        
df = make_adp_df()

replacement_players = {
    'RB':'',
    'WR':'',
    'TE':'',
    'QB':''
}

for _, row in df[:100].iterrows():
    position = row['POS']
    player = row['PLAYER']
    replacement_players[position] = player
    
replacement_players

BASE_URL = 'https://www.fantasypros.com/nfl/projections/{position}.php?week=draft'

def make_projection_df():
    
    #make empty df to add to
    final_df = pd.DataFrame()
    
    for position in ['rb', 'qb', 'te', 'wr']:
        
        res = requests.get(BASE_URL.format(position=position))
        if res.ok:
            soup = BS(res.content, 'html.parser')
            table = soup.find('table', {'id': 'data'})
            df = pd.read_html(str(table))[0]
            
            df.columns = df.columns.droplevel(level=0)
            df['PLAYER'] = df['Player'].apply(lambda x: ''.join(x.split()[:-1]))
            
            if 'REC' in df.columns:
                df['FPTS'] = df['FPTS'] + df['REC']*0.5
                
            df['POS'] = position.upper()
            
            df = df[['PLAYER', 'POS', 'FPTS']]
            final_df = pd.concat([final_df, df])
            
        else:
            print('something didn\'t work', res.status_code)
            return
        
    final_df = final_df.sort_values(by='FPTS', ascending=False)
    
    return final_df

df = make_projection_df()
df.head()

replacement_values = {
    'RB':0,
    'WR':0,
    'QB':0,
    'TE':0
}

for position, player in replacement_players.items():
    if position in ['QB','WR','TE','RB']:
        replacement_values[position]=df.loc[df['PLAYER']==player].values[0,-1]


replacement_values

df['VOR'] = df.apply(lambda row: row['FPTS'] - replacement_values.get(row['POS']), axis=1)
df.head()

df = df.sort_values(by='VOR', ascending=False)
df['VALUERANK'] = df['VOR'].rank(ascending=False)
pd.set_option('display.max_rows', 100)
df.head(100)



Output after reading the html:

    Rank             Player Team (Bye)  POS  Yahoo  Fantrax  AVG
0     1  Christian McCaffrey CAR (13)  RB1    1.0        1  1.0
1     2       Saquon Barkley NYG (11)  RB2    2.0        2  2.0
2     3      Ezekiel Elliott DAL (10)  RB3    3.0        3  3.0
3     4         Michael Thomas NO (6)  WR1    6.0        4  5.0
4     5           Alvin Kamara NO (6)  RB4    4.0        6  5.0 

Output after filtering:

               Player Team (Bye)  POS  AVG
0  Christian McCaffrey CAR (13)  RB1  1.0
1       Saquon Barkley NYG (11)  RB2  2.0
2      Ezekiel Elliott DAL (10)  RB3  3.0
3         Michael Thomas NO (6)  WR1  5.0
4           Alvin Kamara NO (6)  RB4  5.0 

Final output: 

                 PLAYER POS  AVG
0  Christian McCaffrey  RB  1.0
1       Saquon Barkley  RB  2.0
2      Ezekiel Elliott  RB  3.0
3       Michael Thomas  WR  5.0
4         Alvin Kamara  RB  5.0


IndexError: index 0 is out of bounds for axis 0 with size 0

{'RB': 'Marlon Mack',
 'WR': 'John Brown',
 'TE': 'Austin Hooper',
 'QB': 'Matthew Stafford',
 'DS': 'San Francisco 49ers'}

,PLAYER,POS,FPTS
0,LamarJackson,QB,355.40
1,PatrickMahomes,QB,342.90
0,ChristianMcCaffrey,RB,316.95
2,DakPrescott,QB,313.70
3,DeshaunWatson,QB,312.80


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
#df['VOR'] = df.apply(lambda row: row['FPTS'] - replacement_values.get(row['POS']), axis=1)
#df.head()